In [1]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

tokenizer = AutoTokenizer.from_pretrained("seeklhy/codes-7b")

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
        "seeklhy/codes-7b",
        quantization_config=quantization_config,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        # load_in_8bit=True,
        # device_map="auto",
        use_cache=True
    )

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
device = model.device

tokens = tokenizer(['Which countries in Europe have at least 3 car manufacturers?'], return_tensors='pt').to(device)

tokens

{'input_ids': tensor([[23822, 24401,   328, 27268,  1159,   821,  6225,   225,    37,  6346,
          2334, 12326, 47810,    49]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [3]:
generated_ids = model.generate(**tokens, max_new_tokens=50, do_sample=False)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/home/colin/Code/USC/544-nlp-project/baseline_models/venv/lib64/python3.12/site-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [4]:
tokenizer.batch_decode(generated_ids)[0]

'Which countries in Europe have at least 3 car manufacturers?\nSELECT country FROM car_manufacturers GROUP BY country HAVING COUNT(country) >= 3;<|endoftext|>'

In [5]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [6]:
import torch

import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)
     

question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question)

In [7]:
print(generated_sql)


SELECT products.name,
       SUM(sales.quantity * products.price) AS revenue
FROM products
JOIN sales ON sales.product_id = products.product_id
JOIN customers ON customers.customer_id = sales.customer_id
JOIN salespeople ON salespeople.salesperson_id = sales.salesperson_id
WHERE salespeople.region = 'New York'
  AND sales.sale_date BETWEEN '2018-01-01' AND '2018-02-01'
GROUP BY products.name
ORDER BY revenue DESC;

[/SQL] ### Explanation This query uses the following joins: - sales.product_id can be joined with products.product_id - sales.customer_id can be joined with customers.customer_id - sales.salesperson_id can be joined with salespeople.salesperson_id - product_suppliers.product_id can be joined with products.product_id The
WHERE clause filters the results to only include sales in the New York region
  and within the last month. The
GROUP BY clause groups the results by product name. The
ORDER BY clause sorts the results by revenue in descending order. ### Sample Output [OUTPUT